# Sub-Canopy Structure Detector

Detects hidden built structures beneath forest canopy using fused Sentinel-1 SAR and
Sentinel-2 optical indicators.  Imagery is streamed from **Microsoft Planetary Computer** --
no account or API key is required.

### Workflow
1. Define your analysis area (shapefile, FGDB layer, bounding box, or place name)
2. Configure date range and analysis parameters
3. Fetch and process imagery
4. Inspect outputs interactively
5. Save results to disk

In [ ]:
# ── Install required packages (run once, then restart the kernel) ───────────────
# Uncomment the line below if you have not installed the package yet.
# !pip install -e ".[dev]" --quiet
#
# Or install dependencies individually:
# !pip install pystac-client planetary-computer stackstac rioxarray xarray dask[array] \
#              geopandas pyogrio fiona shapely geopy scikit-image scipy \
#              matplotlib folium branca Pillow pyproj ipywidgets --quiet

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

import sys, os
# If running the notebook from the repo root, make the src package discoverable
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('__file__')), 'src'))

from sub_canopy_detector.aoi      import AOIBuilder
from sub_canopy_detector.fetcher   import ImageryFetcher
from sub_canopy_detector.analysis  import SubCanopyAnalyser
from sub_canopy_detector.export    import OutputWriter
from sub_canopy_detector.viz       import ResultVisualiser

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

print('Imports OK')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
#  CONFIGURATION -- edit this cell to set up your analysis
# ═══════════════════════════════════════════════════════════════════════

STUDY_NAME   = 'my_study'          # prefix for all output filenames
OUTPUT_DIR   = './outputs'         # directory where results are saved

# ── Date range ──────────────────────────────────────────────────────────
START_DATE       = '2022-01-01'
END_DATE         = '2024-12-31'
MAX_CLOUD_COVER  = 15              # percent (0-100) -- for Sentinel-2
ORBIT_DIRECTION  = 'ascending'     # 'ascending' or 'descending'

# ── AOI -- uncomment ONE method ─────────────────────────────────────────
AOI_METHOD = 'place_name'          # options: 'place_name', 'bbox', 'polygon', 'shapefile', 'fgdb'

# Method 1: geocode a place name and buffer it
PLACE_NAME   = 'Peten, Guatemala'
BUFFER_KM    = 5.0

# Method 2: bounding box [min_lon, min_lat, max_lon, max_lat]
BBOX = [-90.20, 17.10, -89.95, 17.30]

# Method 3: polygon -- list of (lon, lat) pairs
POLYGON_COORDS = [
    (-90.20, 17.10), (-89.95, 17.10),
    (-89.95, 17.30), (-90.20, 17.30),
]

# Method 4: path to a shapefile or GeoPackage
SHAPEFILE_PATH = r'path/to/your/file.shp'

# Method 5: FileGDB layer
FGDB_PATH  = r'path/to/your/file.gdb'
FGDB_LAYER = 'YourLayerName'

# ── Analysis thresholds (mirrors GEE script defaults) ───────────────────
FOREST_NDVI_THRESHOLD = 0.55
WATER_NDWI_THRESHOLD  = 0.15
STABILITY_FLOOR       = 0.70
TEXTURE_KERNEL_RADIUS = 3          # pixels
ANOMALY_KERNEL_RADIUS = 15         # pixels (Gaussian sigma)
ANOMALY_SIGMA         = 1.5
SLOPE_THRESHOLD       = 15         # degrees
POL_RATIO_MIN         = 0.02
POL_RATIO_MAX         = 0.30

# ── Fusion weights (must sum to 1.0) ────────────────────────────────────
W_STABILITY = 0.30
W_POLRATIO  = 0.20
W_TEXTURE   = 0.20
W_ANOMALY   = 0.20
W_OPTICAL   = 0.10

# ── Output thresholds ───────────────────────────────────────────────────
THRESH_HIGH          = 0.65
THRESH_MEDIUM        = 0.45
MIN_FOOTPRINT_AREA   = 80          # m2

print('Configuration loaded.')

In [ ]:
# ── Define Analysis Area ────────────────────────────────────────────────
if AOI_METHOD == 'place_name':
    aoi = AOIBuilder.from_place_name(PLACE_NAME, buffer_km=BUFFER_KM)

elif AOI_METHOD == 'bbox':
    aoi = AOIBuilder.from_bbox(*BBOX)                 # unpack [min_lon, min_lat, max_lon, max_lat]

elif AOI_METHOD == 'polygon':
    aoi = AOIBuilder.from_polygon(POLYGON_COORDS)

elif AOI_METHOD == 'shapefile':
    aoi = AOIBuilder.from_shapefile(SHAPEFILE_PATH)

elif AOI_METHOD == 'fgdb':
    aoi = AOIBuilder.from_fgdb(FGDB_PATH, layer=FGDB_LAYER)

else:
    raise ValueError(f"Unknown AOI_METHOD '{AOI_METHOD}'.")

AOIBuilder.summarise(aoi)

In [ ]:
# ── Quick AOI location check (static matplotlib map) ────────────────────
import matplotlib.patches as mpatches

b = aoi.bbox_wgs84
fig, ax = plt.subplots(figsize=(5, 5))
aoi.gdf_wgs84.boundary.plot(ax=ax, color='#3a86ff', linewidth=2)
aoi.gdf_wgs84.plot(ax=ax, alpha=0.15, color='#3a86ff')
ax.set_title(f'AOI: {aoi.label}', fontsize=10)
ax.set_xlabel('Longitude'); ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

In [ ]:
# ── Fetch Imagery from Planetary Computer ───────────────────────────────
# This queries the STAC API and builds lazy (dask-backed) stacks.
# No large downloads happen here -- data is only read when .compute() is called.

fetcher = ImageryFetcher(
    aoi=aoi,
    start_date=START_DATE,
    end_date=END_DATE,
    max_cloud_cover=MAX_CLOUD_COVER,
    orbit_direction=ORBIT_DIRECTION,
)

imagery = fetcher.fetch_all(verbose=True)
print(imagery)

In [ ]:
# ── Preview scene counts ─────────────────────────────────────────────────
import pandas as pd

s1_times = pd.DatetimeIndex(imagery.s1.time.values)
s2_times = pd.DatetimeIndex(imagery.s2.time.values)

print(f'S1 scenes in stack : {len(s1_times)}')
print(f'  First : {s1_times.min().date()}')
print(f'  Last  : {s1_times.max().date()}')
print(f'S2 scenes in stack : {len(s2_times)}')

In [ ]:
# ── Run Analysis (this triggers actual dask computation) ─────────────────
# Typical runtime for a 10x10 km AOI with 60 S1 scenes: 3-8 minutes.
# Large AOIs will take proportionally longer.

analyser = SubCanopyAnalyser(
    aoi=aoi,
    imagery=imagery,
    # Override any default parameter here, e.g.:
    forest_ndvi_threshold=FOREST_NDVI_THRESHOLD,
    water_ndwi_threshold=WATER_NDWI_THRESHOLD,
    stability_floor=STABILITY_FLOOR,
    texture_kernel_radius=TEXTURE_KERNEL_RADIUS,
    anomaly_kernel_radius=ANOMALY_KERNEL_RADIUS,
    anomaly_sigma=ANOMALY_SIGMA,
    slope_threshold=SLOPE_THRESHOLD,
    pol_ratio_min=POL_RATIO_MIN,
    pol_ratio_max=POL_RATIO_MAX,
    w_stability=W_STABILITY,
    w_polratio=W_POLRATIO,
    w_texture=W_TEXTURE,
    w_anomaly=W_ANOMALY,
    w_optical=W_OPTICAL,
    thresh_high=THRESH_HIGH,
    thresh_medium=THRESH_MEDIUM,
    min_footprint_area=MIN_FOOTPRINT_AREA,
)

result = analyser.run(verbose=True)

In [ ]:
# ── Indicator Breakdown (6-panel figure) ─────────────────────────────────
vis = ResultVisualiser(result=result, aoi=aoi, s1_stack=imagery.s1)
fig = vis.indicator_panel()
plt.show()

In [ ]:
# ── Interactive Map (Leaflet via folium) ──────────────────────────────────
# The map is rendered inline.  Layers can be toggled in the top-right corner.

m = vis.folium_map(
    show_probability=True,
    show_confidence=True,
    show_footprints=True,
)
m

In [ ]:
# ── SAR Time-Series Chart ─────────────────────────────────────────────────
# Shows mean VV backscatter (dB) over the AOI through time.
# high_only=True restricts the spatial mean to confirmed detections.

fig = vis.sar_timeseries_chart(high_only=True)
plt.show()

In [ ]:
# ── Probability Distribution ──────────────────────────────────────────────
fig = vis.probability_histogram()
plt.show()

In [ ]:
# ── Footprint Attribute Table ─────────────────────────────────────────────
fp = result.footprints.copy()
if not fp.empty:
    fp_display = fp.drop(columns='geometry', errors='ignore')
    fp_display = fp_display.sort_values('prob_mean', ascending=False)
    print(f'{len(fp_display)} footprints detected')
    display(fp_display.head(20))
else:
    print('No footprints detected.  Try lowering THRESH_MEDIUM or MIN_FOOTPRINT_AREA.')

In [ ]:
# ── Save All Outputs ──────────────────────────────────────────────────────
# Writes GeoTIFFs, GeoJSON footprints, CSV, and PNG summary.

writer = OutputWriter(
    result=result,
    aoi=aoi,
    output_dir=OUTPUT_DIR,
    study_name=STUDY_NAME,
)

saved_paths = writer.save_all(fmt_vector='geojson', verbose=True)

print('\nSaved files:')
for k, v in saved_paths.items():
    print(f'  {k:20s}: {v}')